# Notebook 4 — Chatbot RAG con memoria (con retrieval y citas) con persistencia de base de datos vectorial (embedding)

Pipeline: **PyPDFLoader → RecursiveCharacterTextSplitter → OpenAIEmbeddings → Chroma → RetrievalQA**

**Objetivo:**

- Construir un Pipeline de Indexación RAG: Demostrar cómo procesar documentos PDF (PyPDFLoader), dividirlos en fragmentos (RecursiveCharacterTextSplitter) y convertirlos en vectores (HuggingFaceEmbeddings) almacenados en una base de datos vectorial (Chroma).

- Implementar Búsqueda Semántica: Enseñar a construir un retriever que encuentre los fragmentos más relevantes basándose en el significado (vectores), como solución a la fragilidad de la búsqueda léxica (difflib) del notebook anterior.

- Demostrar "Grounding" y Citas: Construir una cadena RAG (LCEL) que fuerce al LLM a basar sus respuestas únicamente en los fragmentos de texto recuperados ({context}) y muestre las fuentes (citas) de donde provino la información.

- Comparar Estrategias de Búsqueda: Introducir y comparar dos métodos de retrieval: la búsqueda por similitud estándar (similarity) y la búsqueda por Relevancia Marginal Máxima (mmr) para obtener resultados más diversos.

- Crear una UI Interactiva Completa: Usar gradio (gr.Blocks) para construir una aplicación web que permita al usuario cargar sus propios PDFs, procesarlos y chatear con ellos, incluyendo un selector para comparar los dos tipos de RAG.

> Requisitos: una API key de OpenAI (`OPENAI_API_KEY`) y 1–3 PDFs cortos del curso para pruebas.

## 1) Inslacación de dependencias


In [1]:
# ⬇️ 1. Instalar dependencias
# (Instala/actualiza todas las bibliotecas necesarias en un solo comando)
!pip install -qU \
    langchain \
    langchain-openai \
    langchain-community \
    langchain-text-splitters \
    chromadb \
    tiktoken \
    pypdf \
    gradio \
    sentence-transformers

# ⬇️ 2. Importar y verificar
import langchain
from langchain_core.runnables import RunnablePassthrough


# --- 1. Importar todas las bibliotecas necesarias ---
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
# ---------------------------------

# Imprime la versión instalada para confirmar que se cargó la más reciente
print(f"LangChain version instalada: {langchain.__version__}")

LangChain version instalada: 1.1.0


In [2]:
import os
from google.colab import userdata

# Set the OPENAI_API_KEY environment variable using the secrets manager
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# --- NUEVO: Montar Google Drive para persistencia ---
from google.colab import drive
# Esto pedirá autorización al ejecutar la celda
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

## 1) Interfaz rápida con Gradio
Aquí una interfaz sencilla para probar preguntas y ver las **citas** a las fuentes.

In [6]:


# --- 3. Función de Indexación (El "Paso 1") ---
# Esta función AHORA CREA AMBAS CADENAS RAG y maneja PERSISTENCIA

def process_files(file_list):
    print("Iniciando procesamiento...")

    # Directorio de persistencia en Google Drive
    persist_directory = "/content/drive/MyDrive/chroma_db_data"

    # Inicializar ChromaDB apuntando al directorio de persistencia
    # Si ya existe, cargará los datos. Si no, se creará uno nuevo/vacío.
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )

    msg_extra = ""

    # Si el usuario subió archivos, los procesamos y AÑADIMOS a la DB existente
    if file_list:
        print(f"Procesando {len(file_list)} archivos nuevos...")
        all_chunks = []
        try:
            for file_obj in file_list:
                print(f"  Cargando: {os.path.basename(file_obj.name)}")
                loader = PyPDFLoader(file_obj.name)
                docs = loader.load()
                splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=120)
                chunks = splitter.split_documents(docs)
                all_chunks.extend(chunks)

            print(f"Total de chunks generados: {len(all_chunks)}")

            # Añadir a la DB existente (incremental)
            print("Añadiendo nuevos documentos a ChromaDB...")
            vectordb.add_documents(documents=all_chunks)
            # Chroma 0.4+ persiste automáticamente

            msg_extra = f"✅ Se añadieron {len(all_chunks)} nuevos fragmentos."

        except Exception as e:
            print(f"Error procesando archivos: {e}")
            return None, f"❌ Error al procesar archivos: {e}"
    else:
        print("No se subieron archivos nuevos. Intentando cargar DB existente...")
        msg_extra = "✅ Usando base de datos existente en Drive."

    # Verificar si la DB tiene datos
    collection_count = vectordb._collection.count()
    print(f"Documentos en la base de datos: {collection_count}")

    if collection_count == 0:
        return None, "⚠️ La base de datos está vacía y no se subieron archivos. Por favor sube PDFs."

    try:
        # --- ¡AQUÍ EMPIEZAN LOS CAMBIOS! ---

        # 3. NUEVO: Definir un prompt para re-escribir la consulta
        rephrase_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "Dada la conversación anterior, genera una consulta de búsqueda (query) independiente para poder encontrar la información y responder a la última pregunta.")
        ])

        # 4. NUEVO: Crear una "cadena de re-escritura"
        # Esta cadena toma el historial y la nueva pregunta, y genera una
        # consulta de búsqueda optimizada (un string).
        query_rewriter_chain = rephrase_prompt | llm | StrOutputParser()

        # 5. NUEVO: Definir el prompt de respuesta (que ahora acepta historial)
        # (El 'prompt' anterior no aceptaba historial, por eso es necesario redefinirlo)
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", "Responde la pregunta del usuario de forma concisa (máximo 3 frases) basándote solo en el siguiente contexto:\n<context>\n{context}\n</context>"),
            MessagesPlaceholder(variable_name="chat_history"), # <-- El historial va aquí
            ("user", "{input}")
        ])

        # 6. MODIFICADO: Crear el Pipeline RAG (Similarity)
        retriever_sim = vectordb.as_retriever(search_type='similarity', search_kwargs={'k': 4})

        qa_sim = (
            {
                # El 'context' ahora se recupera usando la consulta RE-ESCRITA
                "context": RunnablePassthrough.assign(
                    standalone_query=query_rewriter_chain # 1. Re-escribe la consulta
                ) | itemgetter("standalone_query") | retriever_sim, # 2. Usa esa consulta para el retriever

                # Pasamos el 'input' (pregunta original) y el 'historial'
                "input": itemgetter("input"),
                "chat_history": itemgetter("chat_history")
            }
            | RunnablePassthrough.assign(
                answer=(answer_prompt | llm) # El prompt de respuesta ahora recibe el historial
            )
        )
        print("✅ Pipeline RAG (Similarity) con memoria listo.")

        # 7. MODIFICADO: Crear el Pipeline RAG (MMR)
        retriever_mmr = vectordb.as_retriever(search_type='mmr', search_kwargs={'k': 4, 'lambda_mult': 0.5})

        qa_mmr = (
             {
                "context": RunnablePassthrough.assign(
                    standalone_query=query_rewriter_chain # 1. Es la misma cadena de re-escritura (creada para similarity)
                ) | itemgetter("standalone_query") | retriever_mmr, # 2. Usa el retriever MMR

                "input": itemgetter("input"),
                "chat_history": itemgetter("chat_history")
            }
            | RunnablePassthrough.assign(
                answer=(answer_prompt | llm) # Usa el mismo prompt de respuesta de similarity
            )
        )
        print("✅ Pipeline RAG (MMR) con memoria listo.")

        # 8. (Igual que antes) Guardar AMBAS -cuando no tenía memoria- cadenas en un diccionario
        all_chains = {
            "RAG (Similarity)": qa_sim,
            "RAG (MMR)": qa_mmr
        }

        return all_chains, f"✅ ¡Éxito! Base de datos cargada ({collection_count} docs). {msg_extra}"

    except Exception as e:
        print(f"Error procesando archivos: {e}")
        return None, f"❌ Error: {e}"


In [7]:
# --- 1. Importar todas las bibliotecas necesarias ---
import gradio as gr
import os
from operator import itemgetter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


# --- 2. Configuración Global (Modelos) ---

# Carga el modelo de embedding gratuito
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Carga el LLM (asume que la API key de OpenAI está en el entorno)
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Plantilla de Prompt (usaremos la misma para ambos RAGs para una comparación justa)
prompt = ChatPromptTemplate.from_template("""Responde la pregunta de forma concisa (máximo 3 frases) basándote solo en este contexto:
<context>
{context}
</context>

Pregunta: {input}""")



def rag_chat(user_message, history, rag_chain_state, rag_type):

    # Comprueba si el pipeline RAG ("state") ha sido creado
    if rag_chain_state is None:
        return "Por favor, sube uno o más archivos PDF y presiona 'Procesar Archivos' primero."

    # --- ¡NUEVO! Convertir el historial de Gradio al formato de LangChain ---
    chat_history_messages = []
    for human_msg, ai_msg in history:
        chat_history_messages.append(HumanMessage(content=human_msg))
        chat_history_messages.append(AIMessage(content=ai_msg))
    # -----------------------------------------------------------------

    # Usa el 'rag_type' para seleccionar la cadena correcta
    try:
        chain_to_use = rag_chain_state[rag_type]
        print(f"Ejecutando con: {rag_type} y {len(chat_history_messages)} mensajes en historial.")

        # --- MODIFICADO: Invoca la cadena con 'input' Y 'chat_history' ---
        out = chain_to_use.invoke({
            'input': user_message,
            'chat_history': chat_history_messages
        })

        # (El resto de la función es igual: extraer 'answer' y 'context')
        ans = out['answer'].content
        srcs = out.get('context', [])

        cites = []
        for d in srcs:
            meta = d.metadata or {}
            src = os.path.basename(meta.get('source','?'))
            page = meta.get('page','?')
            cites.append(f"{src} (pág. {page})")

        footer = '\n\nFuentes:\n- ' + '\n- '.join(dict.fromkeys(cites)) if cites else ''
        return ans + footer

    except Exception as e:
        print(f"Error en 'rag_chat': {e}")
        return f"Error al generar respuesta: {e}"


# --- 5. Construir la Interfaz de Gradio con "Blocks" ---

with gr.Blocks(title="Chatbot RAG con Carga y Selector") as demo:

    # 'gr.State' es la variable "invisible" que guarda el diccionario de pipelines RAG
    rag_chain_state = gr.State(None)

    gr.Markdown("# Chatbot RAG: Carga de Archivos y Comparador (Similarity vs. MMR)")
    gr.Markdown("Sube tus PDFs, presiona 'Procesar Archivos' y luego chatea con tus documentos.")

    with gr.Row():
        with gr.Column(scale=1):
            # Componente de carga de archivos
            file_uploader = gr.File(
                label="Sube tus PDFs",
                file_count="multiple",
                file_types=[".pdf"]
            )

            # ¡NUEVO! Selector de tipo de RAG
            rag_selector = gr.Radio(
                ["RAG (Similarity)", "RAG (MMR)"],
                label="Elige el tipo de Retriever",
                value="RAG (Similarity)" # Valor por defecto
            )

            # Botón para iniciar la indexación
            process_button = gr.Button("Procesar Archivos", variant="primary")

            # Caja de estado para mensajes de éxito o error
            status_box = gr.Textbox(label="Estado", interactive=False)

        with gr.Column(scale=2):
            # Interfaz de chat estándar
            chat_interface = gr.ChatInterface(
                fn=rag_chat,
                # Pasa AMBAS entradas adicionales a la función de chat
                additional_inputs=[rag_chain_state, rag_selector]
            )

    # --- 6. Conectar los Componentes ---

    # Conecta el botón 'process_button' a la función 'process_files'
    process_button.click(
        fn=process_files,
        inputs=[file_uploader],
        outputs=[rag_chain_state, status_box] # La salida se guarda en el estado y el status
    )

print("Lanzando interfaz de Gradio...")
demo.launch(share=False, debug=True)

/tmp/ipython-input-2465799915.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Lanzando interfaz de Gradio...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## 8) Buenas prácticas y notas finales
- **Citación obligatoria**: siempre devolver fuentes (archivo y página) para confianza y auditoría.
- **Evaluación**: verifica que los fragmentos recuperados contienen evidencia suficiente.
- **Tamaño de chunk**: prueba 400–900 tokens; solape 10–20%.
- **k y search_type**: ajusta según el corpus. `mmr` puede mejorar diversidad.
- **Persistencia**: usa `persist_directory` para reutilizar índices.
- **Privacidad**: no subas documentos sensibles a Colab sin autorización.